# DATA MINING PROJECT: Analysis of a Supermarket’s Customers
## 4) Pattern Mining
### *Antonio Strippoli, Valerio Mariani*

In [ ]:
from functions import *  # Custom function for the analysis
from gsp import apriori
import datetime
import logging
import time
import os

# Set logging
logging.basicConfig(level=logging.INFO, filename="log.txt", filemode="a+", format="%(message)s")
logging.getLogger().addHandler(logging.StreamHandler())

### Apply GSP on sequential data

In [ ]:
# Main cycle: apply GSP multiple times
params = {
    'min_sup': [0.4, 0.35, 0.3, 0.25, 0.2, 0.15],
    'min_baskets': [20, 10, 5, 3, 2],
}
for min_sup in params['min_sup']:
    for min_baskets in params['min_baskets']:
        logging.info(f"MIN_BASKETS: {min_baskets}, MIN_SUP: {min_sup}")

        # Read the dataset
        df = read_dataset()
        # Remove some baskets
        df = remove_baskets(df, min_baskets)
        # Convert into seq form
        seq_data = sequentialize(df)
        
        # Apply GSP
        t0 = time.time()
        result_set = apriori(seq_data, min_sup, verbose=False)
        t1 = time.time()

        # Compute n. of sequences with len > 2 and n. of sequences containing duplicates
        cnt_len_2 = 0
        cnt_duplicates = 0
        for r in result_set:
            r = r[0]
            tmp = []
            for l in r:
                tmp.extend(l)
            if len(tmp) >= 2:
                cnt_len_2 += 1
                if len(set(tmp)) < len(tmp):
                    cnt_duplicates += 1

        logging.info(
            f"TOTAL TIME:\t{round(t1-t0, 2)} s\n"\
            f"LEN RESULT SET:\t{len(result_set)}\n"\
            f"LEN SEQ > 2:\t{cnt_len_2}\nN. DUPLICATES:\t{cnt_duplicates}\n"
        )

        # Save
        read_write_result(False, min_baskets, min_sup, result_set)

### Analyze results and collect statistics

In [ ]:
# Config (which result do we want to analyze)
min_baskets = 10
min_sup = 0.25

# Read result
result_set = read_write_result(True, min_baskets, min_sup)
result_set = convert_tuples_to_list(result_set)
print_distribution(result_set)

# Read and prepare the dataset
df = read_dataset()
df = remove_baskets(df, min_baskets)

# Compute mean qta values
result_set = compute_patterns_mean_qta(result_set, df)

# Convert ProdID to ProdDescr
result_set = prodID_to_prodDescr(result_set, df)

result_set